## 函數庫

In [10]:
import os 
import glob
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

data = pd.read_csv(r"D:\Research\clean_data\prepare.csv")
data["全時間格式進入時間"] = pd.to_datetime(data["全時間格式進入時間"],errors = "coerce")
data["全時間格式出場時間"] = pd.to_datetime(data["全時間格式出場時間"],errors = "coerce")
data["進入日"] = pd.to_datetime(data["進入日"], errors="coerce")
data["出場日"] = pd.to_datetime(data["出場日"], errors="coerce")

print(data.head())

        車號      票種      子場站 進出及付費狀態  校正狀態    進站設備    出站設備        進入日  \
0  KLC0802     臨停車  光復校區停車場     已出站  比對一樣  北大門入口外  北大門出口內 2024-10-15   
1  AMG3703  學生計次汽車  光復校區停車場     已出站  比對一樣  南大門入口外  北大門出口內 2024-10-15   
2   1060MX   教職員汽車  光復校區停車場     已出站  比對一樣  北大門入口外  北大門出口外 2024-10-15   
3   9826TU   教職員汽車  光復校區停車場     已出站  比對一樣  北大門入口外  北大門出口外 2024-10-15   
4  KLN5097     臨停車  光復校區停車場     已出站  比對一樣  北大門入口外  北大門出口內 2024-10-15   

       進入時間        出場日      出場時間   停留時數 計價代碼                 紀錄時間  \
0  23:41:25 2024-10-15  23:44:42   0.05  NaN  2024-10-15 23:44:42   
1  23:40:38 2024-10-17  20:22:00  44.69  F11  2024-10-17 20:22:35   
2  23:39:43 2024-10-16  11:07:00  11.45  L12  2024-10-16 11:07:46   
3  23:39:19 2024-10-16  02:05:00   2.43  L12  2024-10-16 02:05:19   
4  23:34:17 2024-10-15  23:44:12   0.17  NaN  2024-10-15 23:44:12   

            全時間格式進入時間           全時間格式出場時間  
0 2024-10-15 23:41:25 2024-10-15 23:44:42  
1 2024-10-15 23:40:38 2024-10-17 20:22:00  
2 2024-10-15 23:39:4

### 篩選工作日

In [11]:
# 國定假日列表
public_holidays = [
    "2024-02-28", # 和平紀念
    "2024-04-04", # 清明
    "2024-04-05", # 清明
    "2024-06-10", # 端午
    "2024-09-17", # 中秋
    "2024-10-10", # 國慶
    "2024-11-20", # 校慶
]
# 清除六日
# 前半年
clean_data1 = data[(data["進入日"] >= "2024-02-17") & (data["進入日"] < "2024-08-01")].copy()
clean_data2 = data[(data["進入日"] >= "2024-09-01") & (data["進入日"] < "2024-12-23")].copy()

# 判斷時間是否位於工作日
clean_data1.loc[:, "entry_weekday"] = clean_data1["進入日"].dt.weekday
clean_data1.loc[:, "exit_weekday"] = clean_data1["出場日"].dt.weekday
clean_data1.loc[:, "entry_holiday"] = clean_data1["全時間格式進入時間"].isin(public_holidays)
clean_data1.loc[:, "exit_holiday"] = clean_data1["全時間格式出場時間"].isin(public_holidays)

clean_data2.loc[:, "entry_weekday"] = clean_data2["進入日"].dt.weekday
clean_data2.loc[:, "exit_weekday"] = clean_data2["出場日"].dt.weekday
clean_data2.loc[:, "entry_holiday"] = clean_data2["全時間格式進入時間"].isin(public_holidays)
clean_data2.loc[:, "exit_holiday"] = clean_data2["全時間格式出場時間"].isin(public_holidays)

# 移除進出時間皆為假日的紀錄
filtered_data1 = clean_data1[~(
    ((clean_data1["entry_weekday"] >= 5) | clean_data1["entry_holiday"]) &  # 進場時間為週末或國定假日
    ((clean_data1["exit_weekday"] >= 5) | clean_data1["exit_holiday"])     # 出場時間為週末或國定假日
)]

filtered_data2 = clean_data2[~(
    ((clean_data2["entry_weekday"] >= 5) | clean_data2["entry_holiday"]) &  # 進場時間為週末或國定假日
    ((clean_data2["exit_weekday"] >= 5) | clean_data2["exit_holiday"])     # 出場時間為週末或國定假日
)]


filtered_data1 = filtered_data1.drop(columns=["entry_weekday", "exit_weekday", "entry_holiday", "exit_holiday","校正狀態","進入日","出場日","進入時間","出場時間","子場站","進站設備","出站設備","計價代碼","紀錄時間"])
filtered_data2 = filtered_data2.drop(columns=["entry_weekday", "exit_weekday", "entry_holiday", "exit_holiday","校正狀態","進入日","出場日","進入時間","出場時間","子場站","進站設備","出站設備","計價代碼","紀錄時間"])

filtered_data1 = filtered_data1[filtered_data1["停留時數"].notna()]
filtered_data1 = filtered_data1[filtered_data1["車號"].str.len().between(4,8)]
filtered_data1.to_csv(r"D:\Research\Report_sheet\工作日統計_8月前.csv",index=False)

filtered_data2 = filtered_data2[filtered_data2["停留時數"].notna()]
filtered_data2 = filtered_data2[filtered_data2["車號"].str.len().between(4,8)]
filtered_data2.to_csv(r"D:\Research\Report_sheet\工作日統計_8月後.csv",index=False)

C:\Users\cody\AppData\Local\Temp\ipykernel_12652\3423961953.py:19: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  clean_data1.loc[:, "entry_holiday"] = clean_data1["全時間格式進入時間"].isin(public_holidays)
C:\Users\cody\AppData\Local\Temp\ipykernel_12652\3423961953.py:20: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  clean_data1.loc[:, "exit_holiday"] = clean_data1["全時間格式出場時間"].isin(public_holidays)
C:\Users\cody\AppData\Local\Temp\ipykernel_12652\3423961953.py:24: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a fut

In [12]:
filtered_data1["是否過夜"] = (filtered_data1["全時間格式進入時間"].dt.hour < 23) & \
                 (filtered_data1["全時間格式出場時間"].dt.hour >= 7) & \
                 (filtered_data1["全時間格式出場時間"].dt.date > filtered_data1["全時間格式進入時間"].dt.date)

filtered_data2["是否過夜"] = (filtered_data2["全時間格式進入時間"].dt.hour < 23) & \
                 (filtered_data2["全時間格式出場時間"].dt.hour >= 7) & \
                 (filtered_data2["全時間格式出場時間"].dt.date > filtered_data2["全時間格式進入時間"].dt.date)


# 計算過夜次數
overnight_counts1 = filtered_data1.groupby("車號")["是否過夜"].sum().reset_index()
overnight_counts2 = filtered_data2.groupby("車號")["是否過夜"].sum().reset_index()

# **獲取每台車的票種**
ticket_types1 = filtered_data1.groupby("車號")["票種"].first().reset_index()
ticket_types2 = filtered_data2.groupby("車號")["票種"].first().reset_index()

# **合併過夜次數與票種資訊**
result_1 = pd.merge(overnight_counts1, ticket_types1, on="車號", how="left")
result_2 = pd.merge(overnight_counts2, ticket_types2, on="車號", how="left")

# **重新排序欄位**
result_1 = result_1[["車號", "票種", "是否過夜"]]
result_1.rename(columns={"是否過夜": "過夜次數"}, inplace=True)

result_2 = result_2[["車號", "票種", "是否過夜"]]
result_2.rename(columns={"是否過夜": "過夜次數"}, inplace=True)

# **輸出 CSV**
result_1.to_csv(r"D:\Research\Report_sheet\過夜次數統計_含票種_8月前.csv", index=False)
result_2.to_csv(r"D:\Research\Report_sheet\過夜次數統計_含票種_8月後.csv", index=False)

# **顯示前幾筆資料**
print(result_1.head())
print(result_2.head())

       車號   票種  過夜次數
0  0000UO  臨停車     1
1  0000VI  臨停車     1
2  0001A8  臨停車     0
3   0001D  臨停車     1
4  0001W7  臨停車     0
       車號   票種  過夜次數
0  0001GQ  臨停車     0
1  0001W7  臨停車     0
2  0002WE  臨停車     0
3  0006C2  臨停車     0
4  0006PJ  臨停車     0


In [13]:
# 設定過夜次數的區間
bins = [0, 9, 19, 29, 39, 49, 59, 69, 79, 89, 99, 999]  # 定義區間
labels = ["0-9", "10-19", "20-29", "30-39", "40-49", "50-59", "60-69", "70-79", "80-89", "90-99", "100+"]

# 使用 cut 函數將過夜次數分組
result_1["過夜次數區間"] = pd.cut(result_1["過夜次數"], bins=bins, labels=labels, right=True)
result_2["過夜次數區間"] = pd.cut(result_2["過夜次數"], bins=bins, labels=labels, right=True)

# 進行分組統計，計算每個票種在各區間中的用戶數
pivot_table1 = result_1.groupby(["票種", "過夜次數區間"]).size().unstack(fill_value=0)
pivot_table1["用戶數"] = pivot_table1.sum(axis=1)
pivot_table1.loc["總計"] = pivot_table1.sum(axis=0)
# 顯示結果
print(pivot_table1)
# 輸出為 CSV
pivot_table1.to_excel(r"D:\Research\Report_sheet\票種過夜次數區間統計_8月前.xlsx")


# 進行分組統計，計算每個票種在各區間中的用戶數
pivot_table2 = result_2.groupby(["票種", "過夜次數區間"]).size().unstack(fill_value=0)
pivot_table2["用戶數"] = pivot_table2.sum(axis=1)
pivot_table2.loc["總計"] = pivot_table2.sum(axis=0)
# 顯示結果
print(pivot_table2)
# 輸出為 CSV
pivot_table2.to_excel(r"D:\Research\Report_sheet\票種過夜次數區間統計_8月後.xlsx")


過夜次數區間       0-9  10-19  20-29  30-39  40-49  50-59  60-69  70-79  80-89  \
票種                                                                         
互惠車輛          29      0      1      0      0      0      0      0      0   
在職專班汽車        13      0      0      0      0      0      0      0      0   
學生機車證          0      0      0      0      0      0      0      0      0   
學生計次汽車       192     52     27     12      7      1      3      1      0   
學生長時汽車        95     57     47     16      6     10      1      1      2   
廠商汽車證         70      7      4      2      3      0      0      0      0   
教職員汽車        383     67     43     25     19      8      2      5      1   
教職員計次          7      0      0      0      0      0      0      0      0   
特殊入校車輛         8      0      0      0      0      0      0      0      0   
臨停車         4271     21      6      2      1      2      1      0      0   
臨時貴賓          53      1      2      1      0      3      0      0      0   
身障優惠        

C:\Users\cody\AppData\Local\Temp\ipykernel_12652\9642751.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pivot_table1 = result_1.groupby(["票種", "過夜次數區間"]).size().unstack(fill_value=0)
C:\Users\cody\AppData\Local\Temp\ipykernel_12652\9642751.py:20: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pivot_table2 = result_2.groupby(["票種", "過夜次數區間"]).size().unstack(fill_value=0)


In [14]:
# 篩選過夜次數 >= 10 的用戶
target_users1 = result_1[result_1["過夜次數"] >= 10]["車號"]
target_users2 = result_2[result_2["過夜次數"] >= 10]["車號"]

# 確保停留時數是數值型態，避免後續計算出錯
filtered_data1["停留時數"] = pd.to_numeric(filtered_data1["停留時數"], errors="coerce")
filtered_data2["停留時數"] = pd.to_numeric(filtered_data2["停留時數"], errors="coerce")

# 4. 篩選出目標用戶的進出紀錄
target_data1 = filtered_data1[filtered_data1["車號"].isin(target_users1)]
target_data2 = filtered_data2[filtered_data2["車號"].isin(target_users2)]


# 5. 按車號計算總停留時數
total_stay_hours1 = target_data1.groupby("車號")["停留時數"].sum().reset_index().round(2)
total_stay_hours2 = target_data2.groupby("車號")["停留時數"].sum().reset_index().round(2)

# 6. 合併票種與過夜次數資訊
final_result1 = pd.merge(total_stay_hours1, result_1[["車號", "票種", "過夜次數"]], on="車號", how="left")
final_result2 = pd.merge(total_stay_hours2, result_2[["車號", "票種", "過夜次數"]], on="車號", how="left")

# 7. 重新排列欄位並命名
final_result1 = final_result1[["車號", "票種", "過夜次數", "停留時數"]]
final_result1.rename(columns={"停留時數": "總停留時數"}, inplace=True)

final_result2 = final_result2[["車號", "票種", "過夜次數", "停留時數"]]
final_result2.rename(columns={"停留時數": "總停留時數"}, inplace=True)

# 8. 輸出結果到 CSV
final_result1.to_csv(r"D:\Research\Report_sheet\過夜10次以上用戶總停留時數_8月前.csv", index=False)
final_result2.to_csv(r"D:\Research\Report_sheet\過夜10次以上用戶總停留時數_8月後.csv", index=False)

# 9. 顯示前幾筆資料
print(final_result1.head())
print(final_result2.head())

       車號      票種  過夜次數    總停留時數
0  0063B7  學生長時汽車    21  1461.51
1  0116LG  學生計次汽車    36   855.66
2  0126VU  學生長時汽車    35  2145.68
3  0315KW   教職員汽車    29  1507.76
4  0436TS   教職員汽車    15  2498.14
       車號      票種  過夜次數    總停留時數
0  0116LG  學生計次汽車    39  1013.02
1  0126VU  學生長時汽車    28  1673.38
2  0436TS   教職員汽車    11  1436.71
3  0497P3  學生長時汽車    57  1482.68
4  0567TS  學生計次汽車    22  2190.48


In [15]:
target_cars1 = final_result1["車號"]
target_cars2 = final_result2["車號"]

filtered_target_data1 = filtered_data1[filtered_data1["車號"].isin(target_cars1)].copy()
filtered_target_data2 = filtered_data2[filtered_data2["車號"].isin(target_cars2)].copy()

filtered_target_data1["月份"] = filtered_target_data1["全時間格式進入時間"].dt.month
filtered_target_data2["月份"] = filtered_target_data2["全時間格式進入時間"].dt.month

monthly_parking_hours1 = filtered_target_data1.groupby(["車號", "票種", "月份"])["停留時數"].sum().unstack(fill_value=0).round(1)
monthly_parking_hours2 = filtered_target_data2.groupby(["車號", "票種", "月份"])["停留時數"].sum().unstack(fill_value=0).round(1)

monthly_parking_hours1.columns = [f"{month}月停車總時數" for month in monthly_parking_hours1.columns]
monthly_parking_hours2.columns = [f"{month}月停車總時數" for month in monthly_parking_hours2.columns]

monthly_parking_hours1["總計停留時數"] = monthly_parking_hours1.sum(axis=1)
monthly_parking_hours2["總計停留時數"] = monthly_parking_hours2.sum(axis=1)

final_table1 = pd.merge(final_result1[["車號", "票種"]], monthly_parking_hours1, on=["車號", "票種"], how="left").fillna(0)
final_table2 = pd.merge(final_result2[["車號", "票種"]], monthly_parking_hours2, on=["車號", "票種"], how="left").fillna(0)




# 儲存為 CSV
final_table1.to_excel(r"D:\Research\Report_sheet\各月份停車總時數_8月前.xlsx", index=False)
final_table2.to_excel(r"D:\Research\Report_sheet\各月份停車總時數_8月後.xlsx", index=False)

print("✅ 完成！車號各月份停車總時數已計算並存檔")



✅ 完成！車號各月份停車總時數已計算並存檔
